In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv("capital_one_seating_price - attempt2.csv") 
print(df.head(5))

       Section  Min Seats  Max Seats  Wizards  Theater resale  Deftones  \
0  Section 100      298.8      365.2    487.0           432.0     250.0   
1  Section 101      298.8      365.2    422.0           432.0     220.0   
2  Section 102      298.8      365.2    363.0           305.0     200.0   
3  Section 103      298.8      365.2    334.0           305.0     235.0   
4  Section 104      298.8      365.2    280.0           299.0     175.0   

   Wizards 2  Capitals  
0      120.0     612.0  
1       91.0     612.0  
2       50.0     695.0  
3       30.0     545.0  
4       30.0     565.0  


In [17]:
#Finding Min and Max Ticket Capacity in Arena According to df 
print(f'Min seats: {df["Min Seats"].values.sum()}')
print(f'Max seats: {df["Max Seats"].values.sum()}')
print("Since the max seating is closer to 21K, we know the min seat estimate is more accurate for our purposes.")

Min seats: 20301.8
Max seats: 25970.699999999997
Since the max seating is closer to 21K, we know the min seat estimate is more accurate for our purposes.


In [18]:
# Seats per section
section_100_seats = []
section_200_seats = []
section_400_seats = []
floor_seats = []
suite_seats_lower = []
suite_seats_kettler = []
suite_seats_lexus = []

for index, row in df.iterrows():
    section = row["Section"]
    seat_count = row["Min Seats"]

    if section.startswith("Section"):
        # Extract the section number (e.g., "Section 104" -> 104)
        try:
            section_number = int(section.split(" ")[1])
        except (IndexError, ValueError):
            continue

        if 100 <= section_number < 200:
            section_100_seats.append(seat_count)
        elif 200 <= section_number < 300:
            section_200_seats.append(seat_count)
        elif 400 <= section_number < 500:
            section_400_seats.append(seat_count)

    elif section.startswith("Floor"):
        floor_seats.append(seat_count)

    elif section == "Lower Level Suite":
        suite_seats_lower.append(seat_count)
    elif section == "Kettler Loft":
        suite_seats_kettler.append(seat_count)
    elif section == "Lexus Level":
        suite_seats_lexus.append(seat_count)

# Combine all suite seats
suite_total = suite_seats_lower + suite_seats_kettler + suite_seats_lexus

# Print results
print(f'Section 100 Total Seats: {sum(section_100_seats)}')
print(f'Section 200 Total Seats: {sum(section_200_seats)}')
print(f'Section 400 Total Seats: {sum(section_400_seats)}')
print(f'Floor Total Seats: {sum(floor_seats)}')
print(f'Suite Total Seats: {sum(suite_total)}')
print(f'  - Lower Level Suite: {sum(suite_seats_lower)}')
print(f'  - Kettler Loft Suite: {sum(suite_seats_kettler)}')
print(f'  - Lexus Level Suite: {sum(suite_seats_lexus)}')

print(f'\nEstimated Total Seats: {sum(section_100_seats) + sum(section_200_seats) + sum(section_400_seats) + sum(floor_seats) + sum(suite_total)}')


Section 100 Total Seats: 6573.6
Section 200 Total Seats: 3735.0
Section 400 Total Seats: 6772.799999999999
Floor Total Seats: 1494.0
Suite Total Seats: 1726.4
  - Lower Level Suite: 581.0
  - Kettler Loft Suite: 896.4000000000001
  - Lexus Level Suite: 249.00000000000003

Estimated Total Seats: 20301.800000000003


In [19]:
def label_section_type(section):
    if section.startswith("Section"):
        return "bowl"
    elif section.startswith("Floor"):
        return "floor"
    elif section in ["Lower Level Suite", "Kettler Loft", "Lexus Level"]:
        return "suite"
    else:
        return "other"

df['Section Type'] = df['Section'].apply(label_section_type)

In [20]:
event_columns = ['Wizards', 'Theater resale', 'Deftones', 'Wizards 2', 'Capitals']
df[event_columns] = df[event_columns].apply(pd.to_numeric, errors='coerce')
df['Section Avg'] = df[event_columns].mean(axis=1)

# Compute new overall average using only bowl & floor
filtered_df = df[df['Section Type'].isin(['bowl', 'floor'])]
overall_avg_filtered = filtered_df['Section Avg'].mean()

# Calculate multiplier using that filtered average
df['Multiplier'] = df['Section Avg'] / overall_avg_filtered

In [21]:
#creates pricing dataset with multiplier and price for each section once base av price is determined

file_path = "capital_one_seating_price - attempt2.csv"  # Make sure this file is in the same directory
df = pd.read_csv(file_path)

# List of event columns to average over
event_columns = ['Wizards', 'Theater resale', 'Deftones', 'Wizards 2', 'Capitals']

# Calculate the average price per section across all events
df['Section Avg'] = df[event_columns].mean(axis=1)

# Calculate overall average to use for normalizing
overall_avg = df['Section Avg'].mean()

# Calculate multiplier for each section
df['Multiplier'] = df['Section Avg'] / overall_avg

# Function to get dynamic prices based on a base ticket price
def get_dynamic_prices(base_price):
    result = df[['Section', 'Multiplier']].copy()
    result['DynamicPrice'] = (result['Multiplier'] * base_price).round(2)
    return result

# Example usage
if __name__ == "__main__":
    # Input: Set your base ticket price here
    base_price = float(input("Enter average ticket price: $"))
    
    dynamic_prices = get_dynamic_prices(base_price)
    print("\nDynamic Pricing by Section:")
    print(dynamic_prices)

    # Optionally save to CSV
    output_file = f"dynamic_prices_${int(base_price)}.csv"
    dynamic_prices.to_csv(output_file, index=False)
    print(f"\nSaved to {output_file}")



Dynamic Pricing by Section:
         Section  Multiplier  DynamicPrice
0    Section 100    1.452603        209.17
1    Section 101    1.357852        195.53
2    Section 102    1.232535        177.49
3    Section 103    1.107218        159.44
4    Section 104    1.030806        148.44
..           ...         ...           ...
195  Lexus Level    0.955157        137.54
196  Lexus Level    0.955157        137.54
197  Lexus Level    0.955157        137.54
198  Lexus Level    0.955157        137.54
199  Lexus Level    0.955157        137.54

[200 rows x 3 columns]

Saved to dynamic_prices_$144.csv


In [22]:
def calculate_total_revenue(base_price):
    # Step 1: Recalculate multipliers
    event_columns = ['Wizards', 'Theater resale', 'Deftones', 'Wizards 2', 'Capitals']
    df[event_columns] = df[event_columns].apply(pd.to_numeric, errors='coerce')
    df['Section Avg'] = df[event_columns].mean(axis=1)
    overall_avg = df['Section Avg'].mean()
    df['Multiplier'] = df['Section Avg'] / overall_avg

    # Step 2: Calculate dynamic price for each section
    df['Dynamic Price'] = df['Multiplier'] * base_price

    # Step 3: Clean up Min Seats
    df['Min Seats'] = pd.to_numeric(df['Min Seats'], errors='coerce')

    # Step 4: Calculate revenue per section
    df['Section Revenue'] = df['Dynamic Price'] * df['Min Seats']

    # Step 5: Sum it all
    total_revenue = df['Section Revenue'].sum()

    return round(total_revenue, 2)


In [24]:
base_price = float(input("Enter average ticket price: $"))
revenue = calculate_total_revenue(base_price)
print(f"\nEstimated Total Revenue from Ticket Sales for A Packed Venue: ${revenue:,.2f}")
print(f"\nEstimated Total Revenue from Ticket Sales for a Concert Venue (3/4 sections available): ${revenue*.75:,.2f}")




Estimated Total Revenue from Ticket Sales for A Packed Venue: $3,268,208.29

Estimated Total Revenue from Ticket Sales for a Concert Venue (3/4 sections available): $2,451,156.22
